# JS: Viewport Height

> JavaScript generator for dynamic viewport height calculation.

In [ ]:
#| default_exp js.viewport

In [ ]:
#| export
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds

## generate_viewport_height_js

Sets explicit pixel height on the card stack to fill available vertical space.
Measures sibling elements, parent padding, and space below to compute the
remaining height. Includes a debounced resize handler.

The `container_id` parameter is the consumer's parent container that holds
the card stack alongside header/footer elements. If not provided, the card
stack measures from its own position.

In [ ]:
#| export
def generate_viewport_height_js(
    ids: CardStackHtmlIds,  # HTML IDs for this card stack instance
    container_id: str = "",  # Consumer's parent container ID (empty = use card stack parent)
) -> str:  # JavaScript code fragment for viewport height calculation
    """Generate JS for dynamic viewport height calculation."""
    # If no container_id, use the card stack's parent element
    container_selector = (
        f"document.getElementById('{container_id}')"
        if container_id
        else f"document.getElementById('{ids.card_stack}').parentElement"
    )

    return f"""
        // === Viewport Height Calculation ===

        function isNonVisualElement(el) {{
            if (!el || el.nodeType !== Node.ELEMENT_NODE) return true;
            const tag = el.tagName;
            if (tag === 'SCRIPT' || tag === 'STYLE') return true;
            if (tag === 'INPUT' && el.type === 'hidden') return true;
            const styles = getComputedStyle(el);
            if (styles.display === 'none') return true;
            return false;
        }}

        function getElementOuterHeight(el) {{
            if (isNonVisualElement(el)) return 0;
            const styles = getComputedStyle(el);
            const marginTop = parseFloat(styles.marginTop) || 0;
            const marginBottom = parseFloat(styles.marginBottom) || 0;
            return el.getBoundingClientRect().height + marginTop + marginBottom;
        }}

        function calculateSpaceBelowContainer(container) {{
            let spaceUsedBelow = 0;
            let currentElement = container;
            let parent = container.parentElement;
            while (parent && parent !== document.body && parent !== document.documentElement) {{
                const parentStyles = getComputedStyle(parent);
                spaceUsedBelow += parseFloat(parentStyles.paddingBottom) || 0;
                let foundCurrent = false;
                for (const sibling of parent.children) {{
                    if (sibling === currentElement) {{ foundCurrent = true; continue; }}
                    if (!foundCurrent) continue;
                    spaceUsedBelow += getElementOuterHeight(sibling);
                }}
                currentElement = parent;
                parent = parent.parentElement;
            }}
            return spaceUsedBelow;
        }}

        function calculateSiblingHeight(container, excludeId) {{
            let height = 0;
            for (const child of container.children) {{
                if (child.id === excludeId) continue;
                height += getElementOuterHeight(child);
            }}
            return height;
        }}

        function calculateAndSetViewportHeight() {{
            const container = {container_selector};
            const cardStack = document.getElementById('{ids.card_stack}');
            const cardStackInner = document.getElementById('{ids.card_stack_inner}');
            if (!container || !cardStack) return 400;

            const windowHeight = window.innerHeight;
            const containerTop = container.getBoundingClientRect().top;
            const containerStyles = getComputedStyle(container);
            const containerPadTop = parseFloat(containerStyles.paddingTop) || 0;
            const containerPadBot = parseFloat(containerStyles.paddingBottom) || 0;
            const siblingHeight = calculateSiblingHeight(container, '{ids.card_stack}');
            const spaceBelow = calculateSpaceBelowContainer(container);

            const viewportHeight = Math.max(200,
                windowHeight - containerTop - containerPadTop - containerPadBot
                - siblingHeight - spaceBelow
            );

            cardStack.style.height = viewportHeight + 'px';
            cardStack.style.maxHeight = viewportHeight + 'px';
            cardStack.style.minHeight = viewportHeight + 'px';
            if (cardStackInner) cardStackInner.style.height = viewportHeight + 'px';

            return viewportHeight;
        }}

        function _debounce(fn, delay) {{
            let tid;
            return function(...args) {{ clearTimeout(tid); tid = setTimeout(() => fn.apply(this, args), delay); }};
        }}

        const _debouncedResize = _debounce(calculateAndSetViewportHeight, 100);

        if (!window._csResizeListener_{ids.prefix.replace('-', '_')}) {{
            window._csResizeListener_{ids.prefix.replace('-', '_')} = true;
            window.addEventListener('resize', _debouncedResize);
        }}

        // Expose on namespace
        ns.recalculateHeight = calculateAndSetViewportHeight;
    """

In [ ]:
# Test that the generator produces valid JS referencing correct IDs
ids = CardStackHtmlIds(prefix="cs0")
js = generate_viewport_height_js(ids, container_id="my-container")
assert "my-container" in js
assert ids.card_stack in js
assert ids.card_stack_inner in js
assert "ns.recalculateHeight" in js
print("Viewport height JS generator tests passed!")

Viewport height JS generator tests passed!


In [ ]:
# Test without container_id (falls back to parent element)
js_no_container = generate_viewport_height_js(ids)
assert ".parentElement" in js_no_container
print("Viewport height JS fallback container test passed!")

Viewport height JS fallback container test passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()